In [1]:
import os
import matplotlib.pyplot  as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image, ImageOps
import numpy as np
import zipfile

In [3]:
train_dir = "../data/external/fruit_images/train"
valid_dir = "../data/external/fruit_images/valid"
extracted_dir = "../data/external/"

In [4]:
def rename_folder(target_folder, old_folder_name, new_folder_name):
    for folder_name in os.listdir(target_folder):
        if folder_name == old_folder_name:
            old_dir_path = os.path.join(target_folder, folder_name)
            new_dir_path = os.path.join(target_folder, new_folder_name)
            os.rename(old_dir_path, new_dir_path)

In [5]:
rename_folder(target_folder=extracted_dir, old_folder_name= 'Processed Images_Fruits', new_folder_name='fruit_images')

fruit_images_dir = os.path.join(extracted_dir, "fruit_images")
rename_folder(target_folder=fruit_images_dir, old_folder_name='Good Quality_Fruits', new_folder_name='good_quality_fruits')
rename_folder(target_folder=fruit_images_dir, old_folder_name='Bad Quality_Fruits', new_folder_name='bad_quality_fruits')
rename_folder(target_folder=fruit_images_dir, old_folder_name='Mixed Qualit_Fruits', new_folder_name='mixed_quality_fruits')

In [6]:
data_dir = "../data/external/fruit_images"

bad_quality_path = data_dir + "/bad_quality_fruits"
good_quality_path = data_dir + "/good_quality_fruits"
mixed_quality_path = data_dir + "/mixed_quality_fruits"

file_paths=[]
labels=[]
height = 0
width = 0 
samples = 0
sample_count = 20

for fruit_quality in [bad_quality_path, good_quality_path, mixed_quality_path]:
    fruit_list = os.listdir(fruit_quality)
    for fruit in fruit_list:
        fruit_path = os.path.join(fruit_quality, fruit)
        image_list = os.listdir(fruit_path)
        for i, img in enumerate(image_list):
            image_path = os.path.join(fruit_path, img)
            if i < sample_count:
                img = plt.imread(image_path)               
                height += img.shape[0]
                width += img.shape[1]
                samples +=1
            file_paths.append(image_path)
            if fruit_quality == mixed_quality_path:
                labels.append(fruit + '_Mixed')
            else:
                labels.append(fruit)
            
fruit_series = pd.Series(file_paths, name='image')
label_series = pd.Series(labels, name='label')
df = pd.concat([fruit_series, label_series], axis=1)

In [7]:
distinct_labels_df = df.drop_duplicates(subset="label")
# Display the first few rows of the DataFrame for distinct labels
print(distinct_labels_df.head(18))

                                                   image              label
0      ../data/external/fruit_images/bad_quality_frui...          Apple_Bad
1141   ../data/external/fruit_images/bad_quality_frui...         Banana_Bad
2228   ../data/external/fruit_images/bad_quality_frui...          Guava_Bad
3357   ../data/external/fruit_images/bad_quality_frui...           Lime_Bad
4442   ../data/external/fruit_images/bad_quality_frui...         Orange_Bad
5601   ../data/external/fruit_images/bad_quality_frui...    Pomegranate_Bad
6788   ../data/external/fruit_images/good_quality_fru...         Apple_Good
7937   ../data/external/fruit_images/good_quality_fru...        Banana_Good
9050   ../data/external/fruit_images/good_quality_fru...         Guava_Good
10202  ../data/external/fruit_images/good_quality_fru...          Lime_Good
11296  ../data/external/fruit_images/good_quality_fru...        Orange_Good
12512  ../data/external/fruit_images/good_quality_fru...   Pomegranate_Good
18452  ../da

In [8]:
def df_information(df_: pd.DataFrame):    
    class_count = len(list(df["label"].unique()))
    print(f"The dataset contains {df_.shape[0]} images.")
    print(f"The dataset contains the following {class_count} distinct classes. \n")

    for fruit_class in list(df["label"].unique()):
        print(fruit_class)

    items_per_class = list(df_["label"].value_counts())
    print(f"\nEach of the above classses contains {items_per_class} images.")

In [9]:
df_information(df)

The dataset contains 19526 images.
The dataset contains the following 18 distinct classes. 

Apple_Bad
Banana_Bad
Guava_Bad
Lime_Bad
Orange_Bad
Pomegranate_Bad
Apple_Good
Banana_Good
Guava_Good
Lime_Good
Orange_Good
Pomegranate_Good
Apple_Mixed
Banana_Mixed
Guava_Mixed
Lemon_Mixed
Orange_Mixed
Pomegranate_Mixed

Each of the above classses contains [5940, 1216, 1187, 1159, 1152, 1149, 1141, 1129, 1113, 1094, 1087, 1085, 285, 278, 148, 125, 125, 113] images.


In [11]:
# Split the DataFrame into train (70%), validation (15%), and test (15%)
train_df, test_and_valid_df = train_test_split(df, test_size=0.2, random_state=42)
valid_df, test_df = train_test_split(test_and_valid_df, test_size=0.5, random_state=42)

# Check the lengths of the resulting DataFrames
print("Train set length:", len(train_df))
print("Validation set length:", len(valid_df))
print("Test set length:", len(test_df))


Train set length: 15620
Validation set length: 1953
Test set length: 1953


In [10]:
def df_trim(df_: pd.DataFrame, desired_samples_per_class: int) -> pd.DataFrame:

    # Create an empty DataFrame to store the trimmed data
    trimmed_train_df = pd.DataFrame(columns=df_.columns)

    # Iterate through each class and select the first 200 samples
    for class_name in df_['label'].unique():
        class_samples = df_[df_['label'] == class_name].head(desired_samples_per_class)
        trimmed_train_df = pd.concat([trimmed_train_df, class_samples])

    # Reset the index of the trimmed DataFrame
    trimmed_train_df.reset_index(drop=True, inplace=True)

    return trimmed_train_df

In [13]:
trimmed_train_df = df_trim(df_= train_df, desired_samples_per_class=300)

df_information(df_= trimmed_train_df)

The dataset contains 4483 images.
The dataset contains the following 18 distinct classes. 

Apple_Bad
Banana_Bad
Guava_Bad
Lime_Bad
Orange_Bad
Pomegranate_Bad
Apple_Good
Banana_Good
Guava_Good
Lime_Good
Orange_Good
Pomegranate_Good
Apple_Mixed
Banana_Mixed
Guava_Mixed
Lemon_Mixed
Orange_Mixed
Pomegranate_Mixed

Each of the above classses contains [300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 241, 228, 110, 109, 102, 93] images.


In [14]:
def classes_with_less_than_n_samples(df_: pd.DataFrame, desired_samples_per_class: int):
    class_counts = df_['label'].value_counts()
    classes_with_less_than_n_samples_list =  class_counts[class_counts < desired_samples_per_class].index.tolist()
    print(f"Classes with less than {desired_samples_per_class} samples : {classes_with_less_than_n_samples_list}")

    return classes_with_less_than_n_samples_list

In [15]:
# Define a data augmentation function
import random

def augment_image(image_path, save_folder):
    os.makedirs(save_folder, exist_ok=True)
    image = Image.open(image_path)
    # Apply random rotation (you can customize the rotation angle)
    angle = np.random.randint(-15, 15)
    augmented_image = image.rotate(angle)

    # Apply horizontal flip with a 50% chance
    if np.random.choice([True, False]):
        augmented_image = ImageOps.mirror(augmented_image)

    if save_folder:
        # Ensure the save folder exists
        os.makedirs(save_folder, exist_ok=True)
        filename = f"AUG_IMG_{np.random.randint(1000)}_{angle}.jpg"
        file_path = os.path.join(save_folder, filename)
        augmented_image.save(file_path)
    
    return file_path

In [16]:
def df_balance(df_: pd.DataFrame, desired_samples_per_class : int) -> pd.DataFrame:
    save_folder = os.path.join(train_dir)
    # Iterate through the classes with fewer than 200 samples
    target_class = classes_with_less_than_n_samples(df_=df_, desired_samples_per_class=200)
    for class_name in target_class:
            class_samples = df_[df_["label"]==class_name].value_counts()
            class_df = df_[df_["label"] == class_name]

            images_to_augment = len(class_samples)
            while images_to_augment < desired_samples_per_class:
                
                # Choose an image from the class
                random_index = random.randint(0, len(class_df) - 1)
                # Get the image file path at the random index
                random_image = class_df.iloc[random_index]["image"]

                # Apply data augmentation to generate a new image
                augmented_image = augment_image(image_path=random_image, save_folder=save_folder)

                # Append the augmented image to your dataset
                new_df = pd.DataFrame({'image': [augmented_image], 'label': [class_name]})
                df_ = pd.concat([df_, new_df], ignore_index=True)
                images_to_augment+=1
            print(f"For class [{class_name}] I have augmented [{desired_samples_per_class-len(class_samples)}] images.")
    return df_

In [17]:
df_balanced = df_balance(trimmed_train_df, 300)

Classes with less than 200 samples : ['Guava_Mixed', 'Orange_Mixed', 'Pomegranate_Mixed', 'Apple_Mixed']
For class [Guava_Mixed] I have augmented [190] images.
For class [Orange_Mixed] I have augmented [191] images.
For class [Pomegranate_Mixed] I have augmented [198] images.
For class [Apple_Mixed] I have augmented [207] images.


In [18]:
df_information(df_balanced)

The dataset contains 5269 images.
The dataset contains the following 18 distinct classes. 

Apple_Bad
Banana_Bad
Guava_Bad
Lime_Bad
Orange_Bad
Pomegranate_Bad
Apple_Good
Banana_Good
Guava_Good
Lime_Good
Orange_Good
Pomegranate_Good
Apple_Mixed
Banana_Mixed
Guava_Mixed
Lemon_Mixed
Orange_Mixed
Pomegranate_Mixed

Each of the above classses contains [300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 241, 228] images.


In [19]:
def resize_image(image_path : os.path, output_path: os.path, new_width : int, new_height: int):
    image = Image.open(image_path)
    resized_image = image.resize((new_width, new_height))
    resized_image.save(output_path)

new_width = 64
new_height = 64

for index, row in df_balanced.iterrows():
    image_path = row["image"]
    image_name = os.path.basename(image_path)
    new_image_path = os.path.join(train_dir, image_name)
    print(image_path, index)
    if os.path.exists(image_path):
        resize_image(image_path=image_path, output_path=new_image_path, new_width=new_width, new_height=new_height)
        row['train_image'] = new_image_path
    else:
        print(f"File not found: {image_path}")

../data/external/fruit_images/bad_quality_fruits\Banana_Bad\IMG_20190910_175634.jpg 0
../data/external/fruit_images/bad_quality_fruits\Banana_Bad\IMG_7767.JPG 1
../data/external/fruit_images/bad_quality_fruits\Banana_Bad\IMG_20190910_175329.jpg 2
../data/external/fruit_images/bad_quality_fruits\Banana_Bad\IMG_20190910_175311_1.jpg 3
../data/external/fruit_images/bad_quality_fruits\Banana_Bad\IMG_7930.JPG 4
../data/external/fruit_images/bad_quality_fruits\Banana_Bad\IMG_20190910_175514_2.jpg 5
../data/external/fruit_images/bad_quality_fruits\Banana_Bad\IMG_7775.JPG 6
../data/external/fruit_images/bad_quality_fruits\Banana_Bad\IMG_7813.JPG 7
../data/external/fruit_images/bad_quality_fruits\Banana_Bad\IMG_20190910_175653.jpg 8
../data/external/fruit_images/bad_quality_fruits\Banana_Bad\IMG_20190910_175501.jpg 9
../data/external/fruit_images/bad_quality_fruits\Banana_Bad\IMG_8053.JPG 10
../data/external/fruit_images/bad_quality_fruits\Banana_Bad\IMG_20190910_175515_1.jpg 11
../data/externa

In [20]:
valid_df

,image,label
1730,../data/external/fruit_images/bad_quality_frui...,Banana_Bad
12301,../data/external/fruit_images/good_quality_fru...,Orange_Good
9857,../data/external/fruit_images/good_quality_fru...,Guava_Good
8157,../data/external/fruit_images/good_quality_fru...,Banana_Good
19320,../data/external/fruit_images/mixed_quality_fr...,Orange_Mixed
...,...,...
5224,../data/external/fruit_images/bad_quality_frui...,Orange_Bad
18615,../data/external/fruit_images/mixed_quality_fr...,Banana_Mixed
17353,../data/external/fruit_images/good_quality_fru...,Pomegranate_Good
10459,../data/external/fruit_images/good_quality_fru...,Lime_Good


In [ ]:
valid_dir = 
for index, row in valid_df.iterrows():
    image_path = row["image"]
    image_name = os.path.basename(image_path)
    new_image_path = os.path.join(train_dir, image_name)
    print(image_path, index)
    if os.path.exists(image_path):
        resize_image(image_path=image_path, output_path=new_image_path, new_width=new_width, new_height=new_height)
        row['train_image'] = new_image_path
    else:
        print(f"File not found: {image_path}")